In [ ]:
!pip install PyPDF2

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import torch
import PyPDF2
import re

# Dummy user database for login authentication
# Initialize users_db outside of the main Gradio launch block
# Check if users_db is already defined to prevent resetting on cell re-execution
if 'users_db' not in globals():
    users_db = {"student1": "pass123", "student2": "abc456"}


# Store quiz attempts and classroom tracking
user_sessions = {}

# Step 1: Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device set to use {device}")

# Step 2: Load model & tokenizer
try:
    model_name = "ibm-granite/granite-3.3-2b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Removed load_in_4bit=True to address bitsandbytes error
    model = AutoModelForCausalLM.from_pretrained(model_name)
    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=700
    )
    print("✅ Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model/tokenizer: {e}")
    generator = None

# Utility function to generate text
def generate_response(prompt):
    if generator is None:
        print("❌ Error in generate_response: Model not loaded.")
        return "❌ Error: Model not loaded."
    try:
        print(f"Generating response for prompt: {prompt[:100]}...")
        response = generator(prompt)
        if response and isinstance(response, list) and len(response) > 0 and "generated_text" in response[0]:
            print("✅ Response generated successfully.")
            return response[0]["generated_text"]
        else:
            print(f"❌ Unexpected response format: {response}")
            return "❌ Error: Unexpected response format."
    except Exception as e:
        print(f"❌ Error during text generation: {e}")
        return f"❌ Error during text generation: {e}"

# Functionality 1: Concept Understanding
def concept_understanding(concept):
    prompt = f"""
Explain the concept of '{concept}' in a simple and clear way that a 15-year-old student can easily understand. Include examples and real-world applications if possible.
"""
    return generate_response(prompt)

# Functionality 2: Language Learning
def language_learning(language):
    prompt = f"""
Teach me the basics of {language} language. Include grammar rules, common vocabulary, and parts of speech.
"""
    return generate_response(prompt)

# Functionality 3: Test Generator from PDF
def generate_test_from_pdf(pdf_file):
    if not pdf_file:
        return "❌ Please upload a PDF file."
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
        if not text:
            return "❌ Could not extract text from PDF."
        prompt = f"""
Generate a set of multiple-choice questions from the following content:

{text}

Format each question like this:
Qn: <question>
A. <option A>
B. <option B>
C. <option C>
D. <option D>
Correct Answer: <correct letter>
"""
        return generate_response(prompt)
    except Exception as e:
        return f"❌ Failed to process PDF or generate test: {e}"

# Quiz Generation
def quiz_generator(topic):
    prompt = f"Generate a 5-question multiple choice quiz on the topic: {topic}. Format each question with options A, B, C, D and indicate the Correct Answer: <letter>."
    quiz_text = generate_response(prompt)
    print(f"Raw Quiz Generator Output: {quiz_text}") # Add print statement for raw output

    # Parse the generated text into a structured format
    questions = []
    # Split the text by question number, more robust pattern
    # This pattern looks for newline followed by a digit (1 or more) followed by a period
    question_blocks = re.split(r'\n\d+\.\s*', quiz_text)

    for block in question_blocks:
        if not block.strip():
            continue

        # Extract question text - look for text before the first option (A., B., etc.)
        question_match = re.match(r'(.*?)(?=\n[A-D]\.\s*|$)', block, re.DOTALL)
        question_text = question_match.group(1).strip() if question_match else "Could not parse question"

        # Extract options - find all lines starting with A., B., C., or D.
        options = re.findall(r'\n([A-D])\.\s*(.*?)(?=\n[A-D]\.\s*|\nCorrect Answer:|$)', block, re.DOTALL)
        options_dict = {opt[0]: opt[1].strip() for opt in options}

        # Extract correct answer
        correct_answer_match = re.search(r'Correct Answer:\s*([A-D])', block)
        correct_answer = correct_answer_match.group(1) if correct_answer_match else "N/A"

        # Only add question if successfully parsed and has options
        if question_text != "Could not parse question" and options_dict:
            questions.append({
                "question": question_text,
                "options": options_dict,
                "correct_answer": correct_answer
            })
    return questions

# Auth Logic
def authenticate(username, password):
    return users_db.get(username) == password

def register_user(new_username, new_password):
    if new_username in users_db:
        return "❌ Username already exists!"
    else:
        users_db[new_username] = new_password
        return "✅ User registered successfully. You can now login."

# Full App Logic
def run_all(username, concept, language, pdf_file, quiz_topic): # Added quiz_topic to inputs
    if username not in user_sessions:
        user_sessions[username] = []

    concept_output = concept_understanding(concept)
    language_output = language_learning(language)

    # Generate test from PDF
    test_pdf_output = generate_test_from_pdf(pdf_file) # Keep as raw text output
    print(f"PDF Test Generator Output: {test_pdf_output}") # Add print statement

    # Generate quiz from topic
    quiz_data = quiz_generator(quiz_topic) # Generate quiz using the new input

    # Format quiz data for display
    formatted_quiz_output = ""
    if quiz_data:
        for i, q in enumerate(quiz_data):
            formatted_quiz_output += f"Q{i+1}: {q['question']}\n"
            for option_key, option_value in q['options'].items():
                formatted_quiz_output += f"{option_key}. {option_value}\n"
            formatted_quiz_output += f"Correct Answer: {q['correct_answer']}\n\n"
    else:
        formatted_quiz_output = "Could not generate quiz."


    # Prepare Gradio outputs - must match the order of outputs in run_btn.click
    outputs = [
        concept_output, # concept_out
        language_output, # language_out
        test_pdf_output, # test_out
        formatted_quiz_output # quiz_out (formatted string for textbox display)
    ]

    return outputs


# Removed the evaluate_quiz function entirely


# Gradio Interface
def login_fn(user, pwd):
    if authenticate(user, pwd):
        return gr.update(visible=True), gr.update(value="Login successful. Welcome!"), user
    else:
        return gr.update(visible=False), gr.update(value="Invalid credentials!"), ""

with gr.Blocks() as interface:
    gr.Markdown("# 👩‍🏫 EduTutor AI: Personalized Learning Platform")

    username_state = gr.State("")
    # Removed quiz_questions_state as topic quiz is removed

    with gr.Tab("Login"):
        login_user = gr.Textbox(label="Username")
        login_pwd = gr.Textbox(label="Password", type="password")
        login_status = gr.Textbox(label="Status")
        login_button = gr.Button("Login")

    with gr.Tab("Register"):
        new_user = gr.Textbox(label="New Username")
        new_pwd = gr.Textbox(label="New Password", type="password")
        register_button = gr.Button("Register")
        registration_status = gr.Textbox(label="Registration Status")
        register_button.click(fn=register_user, inputs=[new_user, new_pwd], outputs=registration_status)


    with gr.Tab("Classroom"):
        with gr.Column(visible=False) as app_ui:
            concept = gr.Textbox(label="Enter Concept (e.g., Generative AI)")
            language = gr.Radio(choices=["English", "Hindi"], label="Choose Language")
            pdf = gr.File(label="Upload PDF")
            quiz_topic = gr.Textbox(label="Enter Topic for Quiz") # Added new input field for quiz topic
            run_btn = gr.Button("Run EduTutor AI")

            concept_out = gr.Textbox(label="Concept Explanation", show_copy_button=True)
            language_out = gr.Textbox(label="Language Learning", show_copy_button=True)
            test_out = gr.Textbox(label="Generated Test from PDF", show_copy_button=True) # Keep PDF test output
            quiz_out = gr.Textbox(label="Generated Quiz", show_copy_button=True) # Added new output field for the quiz


            # Removed quiz_topic_section and its components


            run_btn.click(fn=run_all,
                         inputs=[username_state, concept, language, pdf, quiz_topic], # Added quiz_topic to inputs
                         outputs=[concept_out, language_out, test_out, quiz_out]) # Added quiz_out to outputs

        login_button.click(fn=login_fn, inputs=[login_user, login_pwd], outputs=[app_ui, login_status, username_state])


interface.launch(debug=True, share=True)

Device set to use cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Model and tokenizer loaded successfully.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eabadb171932e54691.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generating response for prompt: 
Explain the concept of 'what is cybersecurity?' in a simple and clear way that a 15-year-old studen...
✅ Response generated successfully.
Generating response for prompt: 
Teach me the basics of English language. Include grammar rules, common vocabulary, and parts of spe...
✅ Response generated successfully.
Generating response for prompt: 
Generate a set of multiple-choice questions from the following content:

 
Page 1 of 3 
 
 
 
 
Vid...
✅ Response generated successfully.
PDF Test Generator Output: 
Generate a set of multiple-choice questions from the following content:

 
Page 1 of 3 
 
 
 
 
Video transcript  
 
How large  language models work 
 
GPT  or generative pre trained transformer is a  large language model or an LLM that can 
generate  human like text and I've been using GPT  in its  various forms for years . In this video , 
we are going to number one , ask what is an LLM ? Number  two, we are going to describe how 
they work . And then n